In [1]:
"""
References:
https://www.pyimagesearch.com/2021/01/19/opencv-bitwise-and-or-xor-and-not/
https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/
"""

'\nReferences:\nhttps://www.pyimagesearch.com/2021/01/19/opencv-bitwise-and-or-xor-and-not/\nhttps://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/\n'

In [2]:
import cv2                                                                                             #import OpenCV2 library for image processing and algorithms
import math                                                                                            #import python's math library
import pickle                                                                                          #import the pickle library to save and load classifiers
import operator                                                                                        #additional efficient pyhton fucntions
import numpy as np                                                                                     #import numpy mathematical library 
import matplotlib.pyplot as plt                                                                        #import matplotlib library for plotting                    
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))                                  #change width of Jupyer Notebook to use the whole window resolution available

In [3]:
# --------------------------------------------------------------------------------------------DEFINITIONS-------------------------------------------------------------------------------------------
def map_value(value, in_low, in_high, out_low, out_high):                                              #create Arduino map() function in python for usage throughout the code
    return out_low + (out_high - out_low) * ((value - in_low) / (in_high - in_low))                    #scale input lowest,input highest range to output lowest,output highest range then return

def to_bw(image):
    output = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return output

def rotate_img(img, angle):
    result = ndimage.rotate(img, angle) 
    return result 

def find_contours(img_cnt, method, sigma = 0.33):
    img_cnt_bw = to_bw(img_cnt)                                                             #convert rescaled center image to black and white channels for post-processing
    blurred = cv2.GaussianBlur(img_cnt_bw, (3, 3), 0)
    th_cnt_value, th_cnt_img = cv2.threshold(blurred,100,200,cv2.THRESH_OTSU)                    #accurate countours, smoother edges compared to regular binary
    kernel_close = np.ones((3, 3), np.uint8)                                                               #higher kernel = less accurate contours
    morph_cnt_img = cv2.morphologyEx(th_cnt_img, cv2.MORPH_CLOSE, kernel_close)                       #erosion + dilute method (internal spaces removal)
    
    v = np.median(morph_cnt_img)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(morph_cnt_img, lower, upper) 
    
    if method == "EXTERNAL":
        contours_img_cnt, hierarchy_img_cnt = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    elif method == "TREE":
        contours_img_cnt, hierarchy_img_cnt = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 
    return contours_img_cnt, hierarchy_img_cnt, edged

def contour_removal_properties(image, min_corners, max_corners, min_ratio, max_ratio, min_height, max_height):
    contours_found, __ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 
    for c, cnt in enumerate(contours_found):                                                           # loop through all the found contours
        perimeter, area, corners = contour_properties(cnt)
        height, width = shape_height_width(cnt)
        if not min_corners < corners < max_corners or min_ratio < area/perimeter < max_ratio or min_height < height < max_height:
            contours_found.pop(c) 
    return contours_found

def contour_removal_distance_centerpoints(contour_space, contour_space_center_x, contour_space_center_y, dist_min):
    for i, cnt in enumerate(contour_space):
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        if ((((cx - contour_space_center_x)**2) + ((contour_space_center_y)**2))**0.5) > dist_min:
            contours_space.pop(i)
    return contour_space

def draw_selected_contour(original, contour_space, contour, R,G,B, thickness):
    drawn_contours_img = cv2.drawContours(original, contour_space[contour], -1, (R,G,B), thickness, cv2.LINE_AA)
    return drawn_contours_img

def yougest_children_img(img, contours_space_cnt, hierarchy_space_cnt):
    hierarchy_space_cnt = hierarchy_space_cnt[0]
    hierarchy_children_img = img.copy()
    children_found = 0
    for component in zip(contours_space_cnt, hierarchy_space_cnt):
        currentContour = component[0]
        currentHierarchy = component[1]
        x,y,w,h = cv2.boundingRect(currentContour)
        pixel_b, pixel_g, pixel_r = img[int((x+w)/2) ][int((y+h)/2)] 
        pixel_bgr = [pixel_b, pixel_g, pixel_r]
        if pixel_bgr != [241, 235, 230]:
            if currentHierarchy[2] < 0:                                                                            # these are the innermost child components            
                cv2.rectangle(hierarchy_children_img ,(x,y),(x+w,y+h),(0,0,255),3)
                children_found += 1
    return int(children_found/3), hierarchy_children_img

def find_center(image):
    contour_bw = to_bw(image)                                                              #convert rescaled center image to black and white channels for post-processing
    thr_value2, th_shape_img = cv2.threshold(contour_bw,127,255,cv2.THRESH_OTSU)                    #accurate countours, smoother edges compared to regular binary
    kernel_close = np.ones((3, 3), np.uint8)                                                               #higher kernel = less accurate contours
    morph_shape_img = cv2.morphologyEx(th_shape_img , cv2.MORPH_CLOSE, kernel_close)                       #erosion + dilute method (internal spaces removal)
    canny_shape_img = cv2.Canny(morph_shape_img, 150, 500)  
    contours_shape, hierarchy_shape = cv2.findContours(canny_shape_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 

    center_identified_img = image.copy()
    M = cv2.moments(contours_shape[1])
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.circle(center_identified_img, (cx, cy), 7, (0, 0, 255), -1)
        cv2.putText(center_identified_img, "CENTER", (cx - 30, cy - 10),
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    return center_identified_img, cx, cy

def contour_extraction(image, contours_space, cnt):
    mask = np.zeros_like(image) # Create mask where white is what we want, black otherwise
    cv2.drawContours(mask, contours_space, cnt, 255, -1) # Draw filled contour in mask
    masked = np.zeros_like(image) # Extract out the object and place into output image
    masked[mask == 255] = image[mask == 255]
    return masked

def contour_bound_crop(img, cnts, cnt):
    [x,y,w,h] = cv2.boundingRect(cnts[cnt])
    box = cv2.rectangle(img.copy(), (x, y), (x + w, y + h), (0,0,255), 2)
    cropped = img[y-10:y+h+10, x-10:x+w+10]
    return cropped, box

def sharpen_image(image, sharpness, threshold):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv2.GaussianBlur(image, (5,5) , 1.0)
    sharpened = float(sharpness + 1) * image - float(sharpness) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

def contour_properties(contour_selected, req):
    perimeter = cv2.arcLength(contour_selected, True)                                                                    # perimeter of contour c (curved length)     
    area = cv2.contourArea(contour_selected)                                                                     
    corners = len(cv2.approxPolyDP(contour_selected, 0.04*perimeter, True))
    if req == "A":
        return area
    elif req == "P":
        return perimeter
    elif req == "C":
        return corners

def shape_ratio(perimeter, area):
    ratio = area/perimeter   
    return ratio

def fit_ellipse(contour_selected):
    ellipse_found = cv2.fitEllipse(contour_selected)                                                                      # fit an ellipse on the contour
    (center, axes, orientation) = ellipse_found                                                            # extract the main parameter
    majoraxis_length = max(axes)                              
    minoraxis_length = min(axes)
    return minoraxis_length, majoraxis_length

def shape_height_width(contour_selected, val):
    minor, major = fit_ellipse(contour_selected)
    if val == "H":
        return major/minor
    if val == "W":
        return minor/major

def contour_masking(morphology, cnts, selected):
    mask = np.zeros_like(morphology)                                                               # Create mask where white is what we want, black otherwise
    cv2.drawContours(mask, cnts, selected, 255, -1)                                       # Draw filled contour in mask
    masked = np.zeros_like(morphology)                                                              #Extract out the object and place into output image
    masked[mask == 255] = morphology[mask == 255]
    return masked

def harris_method_corners(image):
    harris_method_BGR_img= image.copy()
    harris_method_bw_img = np.float32(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    dst = cv2.cornerHarris(harris_method_bw_img,5,3,0.04)
    ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    harris_corners = cv2.cornerSubPix(harris_method_bw_img,np.float32(centroids),(5,5),(-1,-1),criteria)
    harris_method_BGR_img[dst>0.1*dst.max()]=[0,0,255]   
    
    for c in range(1, len(harris_corners)):
        cv2.circle(harris_method_BGR_img, (int(harris_corners[c,0]), int(harris_corners[c,1])), 7, (0,255,0), 2)
    
    return c, harris_method_BGR_img

def shi_tomasi_method_corners(image):
    shi_tomasi_BGR_img= image.copy()
    shi_tomasi_method_bw_img = cv2.cvtColor(shi_tomasi_BGR_img, cv2.COLOR_BGR2GRAY)
    shi_tomasi_corners = cv2.goodFeaturesToTrack(shi_tomasi_method_bw_img, 0, 0.25, 0.05)
    shi_tomasi_corners = np.int0(shi_tomasi_corners)
                                                                                                  
    for i in shi_tomasi_corners:                                                                                    # draw red color circles on all corners
        x, y = i.ravel()
        cv2.circle(shi_tomasi_BGR_img, (x, y), 3, (0, 0, 255), -1)
    return len(shi_tomasi_corners), shi_tomasi_BGR_img

def houghcircles(image, min_radius, max_radius, circle_dist):
    ___, center_x, center_y = find_center(image)
    circles_count = 0   
    houghcircles_BGR_img= image.copy()
    houghcircles_bw_img = cv2.cvtColor(houghcircles_BGR_img, cv2.COLOR_BGR2GRAY)
    rows = houghcircles_bw_img.shape[0]
    circles = cv2.HoughCircles(houghcircles_bw_img, cv2.HOUGH_GRADIENT, 1, rows / circle_dist,
                               param1=100, param2=30, minRadius=min_radius, maxRadius=max_radius)
    
    dist_center_circle = 0
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1]) 
            dist_center_circle += int(calculateDistance(center[0],center[1],center_x,center_y))
            cv2.circle(houghcircles_BGR_img, center, 1, (0, 100, 100), 3)                           # circle center           
            radius = i[2]                                                                              # circle outline
            cv2.circle(houghcircles_BGR_img, center, radius, (255, 0, 255), 3)
        circles_count = int(circles.size/3)
    return circles_count, houghcircles_BGR_img, dist_center_circle

def houghlines(image, canny_edges):
    lines_img = image.copy()
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 10  # angular resolution in radians of the Hough grid
    threshold = 17 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 15  # minimum number of pixels making up a line
    max_line_gap = 18 # maximum gap in pixels between connectable line segments
    line_image = np.copy(image) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(canny_edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    line_counter = 0
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                line_counter += 1
                cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),5)

        lines_edges = cv2.addWeighted(lines_img, 1, line_image, 1, 0)

        parallel = []
        for i in range(len(lines)):
            for j in range(len(lines)):
                if (i == j):continue
                if (abs(lines[i][0][1] - lines[j][0][1]) == 0):         
                    parallel.append((i,j))
        return line_counter, lines_edges, len(parallel)
    
def center_shape(img_frame, b, g, r):
    mask = np.zeros_like(img_frame)
    rows, cols,_ = mask.shape
    center_x = int(rows/2)
    center_y = int(cols/2)
    center = (center_y,center_x)

    mask = cv2.ellipse(mask, center, axes=(40, 60), angle=24, startAngle=0, endAngle = 360, color=(255,255,255), thickness = -1)
    ellipse_masked_img = np.bitwise_and(img_frame, mask)
    ellipse_masked_img[np.all(ellipse_masked_img == (0, 0, 0), axis=-1)] = (b, g, r)

    return ellipse_masked_img

In [6]:
def process_frame(cntr, colour_type):
    try:
        x, y, w, h = cv2.boundingRect(cntr)                                                     #find the x,y of origin, width and height of the contour
        processing_frame = frame[y:y+h, x:x+w]
        b,g,r = (frame[int((y+h+270)/2), int((x+w+70)/2)])
        center_frame = center_shape(processing_frame, b,g,r) 
                
        contours_uno, hierarchy_uno, canny_uno_img = find_contours(processing_frame, "TREE")
        contours_center, hierarchy_center, canny_center_img = find_contours(center_frame, "EXTERNAL")
    
        contours_final = list()
        hierarchy_center = hierarchy_center[0]
        cut = 0
        for component in zip(contours_center, hierarchy_center):
            currentContour = component[0]
            currentHierarchy = component[1]
            if currentHierarchy[2] < 0:
                contours_final.append(currentContour)
                cut += 1

        #defining all features
        height_found, width_found, area_found, perimeter_found, poly_corners_found = 0,0,0,0,0
        for c in range(len(contours_final)):
            area_found += contour_properties(contours_final[c], "A")
            perimeter_found += contour_properties(contours_final[c], "P")
            poly_corners_found += contour_properties(contours_final[c], "C")
            height_found += shape_height_width(contours_final[c], "H") 
            width_found += shape_height_width(contours_final[c], "W") 

        ratio_found = shape_ratio(perimeter_found, area_found)    
        harris_corners, harris_corners_img = harris_method_corners(center_frame)
        shi_tomasi_corners, shi_tomasi_img = shi_tomasi_method_corners(center_frame)
        lines_found, lines_found_image, parallel_instances_found = houghlines(center_frame, canny_center_img)
        circles_found, houghcircles_img, distance_circles = houghcircles(center_frame, 20, 30, 8)
        children_found_counter, children_found_img = yougest_children_img(processing_frame, contours_uno, hierarchy_uno)

        features = [int(-(-ratio_found // 1)), float("{:.1f}".format(height_found)), float("{:.1f}".format(width_found)), 
                    children_found_counter, lines_found, parallel_instances_found, circles_found, distance_circles, harris_corners, shi_tomasi_corners, poly_corners_found]
        features_array = (np.array(features)).reshape(1, -1)

        result = int(rfc.predict(features_array))
        text = colour_type + str(result) + " UNO CARD"                                                            #if area is higher than area_sensitivity (perfect value for my camera = 8000)
        return text, canny_center_img, x, y, w, h
    
    except (TypeError, IndexError, ZeroDivisionError, cv2.error) as e:
        pass


In [8]:
#--------------------------------------------------------------------------------------------CAMERA SETUP-------------------------------------------------------------------------------------------
camera = cv2.VideoCapture(0)                                                                           #setting camera 0 as our image input
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 800)                                                              #setting camera width resolution as 800
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)                                                             #setting camera height resolution as 480
area_sensitivity = 8000                                                                                #area threshold to detect items

rfc = pickle.load(open("randomforest_clf_optimised.p", "rb"))                                    #load trained model with the pickle library
#---------------------------------------------------------------------------------------CAMERA INPUT PROCESSING-------------------------------------------------------------------------------------
while True:                                                                                            #create a while loop to check for camera input
    _, frame = camera.read()                                                                           #reading the video from the camera in image frames
    #frame = cv2.flip(frame,1)
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)                                                  #convert the colour space from BGR to HSV for further processing
    kernel = np.ones((5,5), "uint8")                                                                   #create a kernel to be used for dilation in order to remove noise/holes from the frame
    
#----------------------------------------------------------------------------------------RED COLOUR PROCESSING-------------------------------------------------------------------------------------
    red_lower = np.array([136, 127, 111], np.uint8)                                                     #set the lower HSV range for the RED colour
    red_upper = np.array([180, 255, 255], np.uint8)                                                    #set the upper HSV range for the RED colour
    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)                                             #create a mask containing only the RED pixels from the original frame
    red_mask = cv2.dilate(red_mask, kernel)                                                            #dilate the red mask and remove the noise by using the kernel
    res_red = cv2.bitwise_and(frame, frame, mask = red_mask)                                           #merge the frames based on the red mask pixel coordinates
    
    contours_red, hierarchy_red = cv2.findContours(red_mask, cv2.RETR_TREE,                            #find the contours on the red mask
                                                   cv2.CHAIN_APPROX_SIMPLE)           
    for i_red, contour_red in enumerate(contours_red):                                                  #go through all the contours
        area_red = cv2.contourArea(contour_red)                                                                #calculate area of the current contour
        if(area_red > area_sensitivity):                                                                   #if area is higher than area_sensitivity (perfect value for my camera = 8000)   
            try:
                result_red, canny_center_red, x_red, y_red, w_red, h_red = process_frame(contour_red, "RED ")
            except TypeError:
                continue
            cv2.imshow("canny_center_red", canny_center_red)
            frame = cv2.rectangle(frame, (x_red, y_red), (x_red + w_red, y_red + h_red), (0, 0, 255), 2)                       #draw a rectangle around the detected image with the RED colour
            cv2.putText(frame, result_red, (x_red, y_red), cv2.FONT_HERSHEY_SIMPLEX,                       #create text "RED UNO CARD" at the origin x,y of the detected contour
                                                   0.7, (0, 0, 255), 2, cv2.LINE_AA)    


#----------------------------------------------------------------------------------------BLUE COLOUR PROCESSING-------------------------------------------------------------------------------------  
    blue_lower = np.array([97, 150, 80], np.uint8)                                                     #set the lower HSV range for the BLUE colour
    blue_upper = np.array([117, 255, 255], np.uint8)                                                   #set the upper HSV range for the BLUE colour
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)                                          #create a mask containing only the BLUE pixels from the original frame
    blue_mask = cv2.dilate(blue_mask, kernel)                                                          #dilate the red mask and remove the noise by using the kernel
    res_blue = cv2.bitwise_and(frame, frame, mask = blue_mask)                                         #merge the frames based on the red mask pixel coordinates
    
    contours_blue, hierarchy_blue = cv2.findContours(blue_mask, cv2.RETR_TREE,                         #find the contours on the blue mask
                                                      cv2.CHAIN_APPROX_SIMPLE)          
    for cnt, contour in enumerate(contours_blue):                                                      #go through all the contours
        area = cv2.contourArea(contour)                                                                #calculate area of the current contour
        if(area > area_sensitivity):                                                                   #if area is higher than area_sensitivity (perfect value for my camera = 8000)
            x, y, w, h = cv2.boundingRect(contour)                                                     #find the x,y of origin, width and height of the contour
            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)                       #draw a rectangle around the detected image with the BLUE colour
            cv2.putText(frame, "BLUE UNO CARD", (x, y), cv2.FONT_HERSHEY_SIMPLEX,                      #create text "BLUE UNO CARD" at the origin x,y of the detected contour
                                                0.7, (255, 0, 0), 2, cv2.LINE_AA)
    
#--------------------------------------------------------------------------------------YELLOW COLOUR PROCESSING-------------------------------------------------------------------------------------   
    yellow_lower = np.array([23, 41, 133], np.uint8)                                                   #set the lower HSV range for the YELLOW colour
    yellow_upper = np.array([23, 150, 255], np.uint8)                                                  #set the upper HSV range for the YELLOW colour
    yellow_mask = cv2.inRange(hsvFrame, yellow_lower, yellow_upper)                                    #create a mask containing only the YELLOW pixels from the original frame
    yellow_mask = cv2.dilate(yellow_mask, kernel)                                                      #dilate the red mask and remove the noise by using the kernel
    res_yellow = cv2.bitwise_and(frame, frame, mask = yellow_mask)                                     #merge the frames based on the red mask pixel coordinates    
            
    contours_yellow, hierarchy_yellow = cv2.findContours(yellow_mask, cv2.RETR_TREE,                   #find the contours on the yellow mask
                                                            cv2.CHAIN_APPROX_SIMPLE)        
    for cnt, contour in enumerate(contours_yellow):                                                    #go through all the contours
        area = cv2.contourArea(contour)                                                                #calculate area of the current contour
        if(area > area_sensitivity):                                                                   #if area is higher than area_sensitivity (perfect value for my camera = 8000)
            x, y, w, h = cv2.boundingRect(contour)                                                     #find the x,y of origin, width and height of the contour             
            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)                     #draw a rectangle around the detected image with the YELLOW colour
            cv2.putText(frame, "YELLOW UNO CARD", (x, y), cv2.FONT_HERSHEY_SIMPLEX,                    #create text "YELLOW UNO CARD" at the origin x,y of the detected contour
                                                0.7, (0, 255, 255), 2, cv2.LINE_AA)      
            
#--------------------------------------------------------------------------------------BLACK COLOUR PROCESSING-------------------------------------------------------------------------------------   
    """
    black_lower = np.array([0, 0, 20], np.uint8)                                                        #set the lower HSV range for the BLACK colour
    black_upper = np.array([180, 255, 60], np.uint8)                                                    #set the upper HSV range for the BLACK colour
    black_mask = cv2.inRange(hsvFrame, black_lower, black_upper)                                        #create a mask containing only the BLACK pixels from the original frame
    black_mask = cv2.dilate(black_mask, kernel)                                                         #dilate the red mask and remove the noise by using the kernel
    res_black = cv2.bitwise_and(frame, frame, mask = black_mask)                                        #merge the frames based on the red mask pixel coordinates    
            
    contours_black, hierarchy_black = cv2.findContours(black_mask, cv2.RETR_TREE,                       #find the contours on the black mask
                                                         cv2.CHAIN_APPROX_SIMPLE)          
    for cnt, contour in enumerate(contours_black):                                                      #go through all the contours
        area = cv2.contourArea(contour)                                                                 #calculate area of the current contour
        if(area > 5000):                                                                                #if area is higher than area_sensitivity (perfect value for my camera = 8000)
            #contours = np.vstack([contours[cnt], contours[cnt]])
            x, y, w, h = cv2.boundingRect(contour)                                                      #find the x,y of origin, width and height of the contour             
            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 0), 2)                          #draw a rectangle around the detected image with the BLACK colour
            cv2.putText(frame, "BLACK UNO CARD", (x, y), cv2.FONT_HERSHEY_SIMPLEX,                      #create text "BLACK UNO CARD" at the origin x,y of the detected contour
                                                   0.7, (0, 0, 0), 2, cv2.LINE_AA)             
    """

#----------------------------------------------------------------------------------------PROGRAM TERMINATION--------------------------------------------------------------------------------------
    cv2.imshow("UNO Card Detection", frame)                                                            #show the camera stream in a window called "UNO Card Detection"
    #cv2.imshow('Red Mask', red_mask)
    #cv2.imshow('Res Red', res_red)
    if cv2.waitKey(10) == ord('q'):                                                                     #if the q key has been pressed then:
        camera.release()                                                                                   #release the camera input
        cv2.destroyAllWindows()                                                                            #close the window
        break                                                                                              #exit the while loop
       

ValueError: not enough values to unpack (expected 3, got 2)